# Kütüphaneleri İçeri Aktarma

In [1]:
# transformers kütüphanesinden pipeline modülünü içeri aktarma
"""
Bu kod, transformers kütüphanesinden pipeline modülünü içeri aktarmaktadır.
Pipeline modülü, doğal dil işleme (NLP) görevlerini kolaylaştırmak için kullanılır.
Bu görevler arasında metin sınıflandırma, duygu analizi, metin oluşturma, çeviri ve daha fazlası bulunmaktadır.
"""
from transformers import pipeline

# Analizciyi işleyecek fonksiyon

In [ ]:
def metin_al() -> str:
    """
    Kullanıcıdan metin girişi alan fonksiyon.
    
    Returns:
        str: Kullanıcının girdiği metin
    """
    return input("\nAnaliz edilecek metni girin: ")
"""
def metin_ozetle(summarizer, metin):
    try:
        if len(metin.split()) < 30:
            return "Metin özetleme için çok kısa"
        sonuc = summarizer(metin, max_length=130, min_length=30, do_sample=False)
        return sonuc
    except Exception as e:
        return f"Hata oluştu: {str(e)}"
"""
def soru_cevapla(qa_pipeline: object, metin: str) -> dict:
    """
    Verilen metin üzerinde soru-cevap işlemi gerçekleştirir.

    Args:
        qa_pipeline (object): Soru-cevap işlemini gerçekleştirecek pipeline nesnesi
        metin (str): Analiz edilecek metin

    Returns:
        dict: Soru-cevap işleminin sonucunu içeren sözlük
        str: Hata durumunda hata mesajı
    """
    try:
        sonuc = qa_pipeline({
            'question': 'Bu metnin ana konusu nedir?',
            'context': metin
        })
        return sonuc
    except Exception as e:
        return f"Hata oluştu: {str(e)}"

def metin_analiz(analyzer: object, metin: str, islem_tipi: str) -> None:
    """
    Verilen metin üzerinde belirtilen analiz işlemini gerçekleştirir ve sonuçları yazdırır.

    Args:
        analyzer (object): Analiz işlemini gerçekleştirecek nesne
        metin (str): Analiz edilecek metin
        islem_tipi (str): Gerçekleştirilecek analiz işleminin tipi ("Soru Cevaplama" veya diğer)

    Returns:
        None
    """
    try:
        print("\nANALİZ SONUCU:")
        print(f"Metin: {metin}")
        
        # İşlem tipine göre uygun fonksiyonu çağır
        if islem_tipi == "Soru Cevaplama":
            sonuc = soru_cevapla(analyzer, metin)
        # Özetleme işlemi şu an devre dışı
        #elif islem_tipi == "Özetleme":
        #    sonuc = metin_ozetle(analyzer, metin)
        else:
            # Diğer analiz işlemleri için doğrudan analyzer'ı kullan
            sonuc = analyzer(metin)
            
        print(f"Sonuç: {sonuc}")
    except Exception as e:
        print(f"Hata oluştu: {str(e)}")
    print("-" * 50)

# huggingface'e giriş yapma

In [3]:
from huggingface_hub import login, whoami # Hugging Face Hub'a giriş yapmak için gerekli fonksiyonlar

# Hugging Face'den aldığınız token'ı buraya yazın
# (https://huggingface.co/settings/tokens adresinden alabilirsiniz)
token = input("Hugging Face token'ınızı girin: ")

# Login işlemi
login(token=token)
user_info = whoami()
print(user_info["fullname"], "olarak giriş yapıldı.")

kayra bulut olarak giriş yapıldı.


# Hazır modellerle pipeline oluşturma

In [4]:
# Modelleri tanımlama
duygu_analizi_modeli = "savasy/bert-base-turkish-sentiment-cased"  # Duygu analizi için model
metin_siniflandirma_modeli = "dbmdz/bert-base-turkish-cased"  # Metin sınıflandırma için model
ner_modeli = "savasy/bert-base-turkish-ner-cased"  # Varlık ismi tanıma için model
qa_modeli = "savasy/bert-base-turkish-squad"  # Soru cevaplama için model
ozetleme_modeli = "mrm8488/bert2bert_shared-turkish-summarization"  # Özetleme için model

In [ ]:
# NER Etiketleri ve Açıklamaları:

# Organizasyon Etiketleri
# B-ORG: "Beginning of Organization" - Bir organizasyon/kuruluş isminin başlangıcını belirtir
# I-ORG: "Inside of Organization" - Organizasyon isminin devam eden kısmını belirtir

# Kişi İsimleri Etiketleri
# B-PER: "Beginning of Person" - Bir kişi isminin başlangıcını belirtir
# I-PER: "Inside of Person" - Kişi isminin devam eden kısmını belirtir

# Lokasyon Etiketleri
# B-LOC: "Beginning of Location" - Bir konum isminin başlangıcını belirtir
# I-LOC: "Inside of Location" - Konum isminin devam eden kısmını belirtir

# Tarih Etiketleri
# B-DATE: "Beginning of Date" - Bir tarih ifadesinin başlangıcını belirtir
# I-DATE: "Inside of Date" - Tarih ifadesinin devam eden kısmını belirtir

# Zaman Etiketleri
# B-TIME: "Beginning of Time" - Bir zaman ifadesinin başlangıcını belirtir
# I-TIME: "Inside of Time" - Zaman ifadesinin devam eden kısmını belirtir

# Para Birimi Etiketleri
# B-MONEY: "Beginning of Money" - Bir para birimi ifadesinin başlangıcını belirtir
# I-MONEY: "Inside of Money" - Para birimi ifadesinin devam eden kısmını belirtir

# Yüzde/Oran Etiketleri
# B-PERCENT: "Beginning of Percentage" - Bir yüzde/oran ifadesinin başlangıcını belirtir
# I-PERCENT: "Inside of Percentage" - Yüzde/oran ifadesinin devam eden kısmını belirtir

# Ürün İsimleri Etiketleri
# B-PROD: "Beginning of Product" - Bir ürün isminin başlangıcını belirtir
# I-PROD: "Inside of Product" - Ürün isminin devam eden kısmını belirtir

# Olay İsimleri Etiketleri
# B-EVENT: "Beginning of Event" - Bir olay isminin başlangıcını belirtir
# I-EVENT: "Inside of Event" - Olay isminin devam eden kısmını belirtir

# Diğer İsimler
# O: "Outside" - Herhangi bir varlık kategorisine ait olmayan kelimeler
# B-MISC: "Beginning of Miscellaneous" - Diğer kategorilere girmeyen özel isimlerin başlangıcı
# I-MISC: "Inside of Miscellaneous" - Diğer kategorilere girmeyen özel isimlerin devamı

In [ ]:
# Pipeline'ları oluşturma
sentiment_analyzer = pipeline("sentiment-analysis", model=duygu_analizi_modeli, return_all_scores=True)
text_classifier = pipeline("text-classification", model=metin_siniflandirma_modeli, return_all_scores=True)
ner_analyzer = pipeline("ner", model=ner_modeli)
qa_pipeline = pipeline("question-answering", model=qa_modeli)
# summarization_pipeline = pipeline("summarization", model=ozetleme_modeli, tokenizer=ozetleme_modeli)

# Pipeline listesini oluşturma
pipeline_list = {
    "Duygu Analizi": sentiment_analyzer,
    "Metin Sınıflandırma": text_classifier,
    "Varlık İsmi Tanıma": ner_analyzer,
    "Soru Cevaplama": qa_pipeline,
    # "Özetleme": summarization_pipeline
}

/home/internet/Desktop/ytu_fark_2024-2025_yz/.venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initiali

In [6]:
while True:
        metin = metin_al()
        
        if metin.lower() == 'q':
            print("\nProgram sonlandırılıyor...")
            break
            
        for i, (name, analyzer) in enumerate(pipeline_list.items(), 1):
            print(f"{i}. {name}")
            metin_analiz(analyzer, metin, name)

1. Duygu Analizi

ANALİZ SONUCU:
Metin:  Dışa aktarma ayarlarını yapılandırın. Bu aşamada, sanal makinenizin hangi bileşenlerinin dahil edileceğini seçebilirsiniz. İleri (Next) butonuna tıklayarak devam edin. Dışa Aktarma İşlemini Tamamlayın: Son adımda, ayarlarınızı gözden geçirin ve Dışa Aktar (Export) butonuna tıklayın. Bu işlem, sanal makinenizi bir OVA dosyası olarak dışa aktaracaktır. Bu adımlar, VirtualBox kullanarak bir OVA dosyası oluşturmanıza yardımcı olacaktır. OVA dosyaları, sanal makinelerin kolayca taşınabilmesi ve başka sistemlerde çalıştırılabilmesi için oldukça kullanışlıdır.
Sonuç: [[{'label': 'negative', 'score': 0.1497812569141388}, {'label': 'positive', 'score': 0.8502187728881836}]]
--------------------------------------------------
2. Metin Sınıflandırma

ANALİZ SONUCU:
Metin:  Dışa aktarma ayarlarını yapılandırın. Bu aşamada, sanal makinenizin hangi bileşenlerinin dahil edileceğini seçebilirsiniz. İleri (Next) butonuna tıklayarak devam edin. Dışa Aktarma İşlemi

/home/internet/Desktop/ytu_fark_2024-2025_yz/.venv/lib/python3.12/site-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Sonuç: {'score': 0.0007658902904950082, 'start': 51, 'end': 107, 'answer': 'sanal makinenizin hangi bileşenlerinin dahil edileceğini'}
--------------------------------------------------

Program sonlandırılıyor...
